In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import substring, length, col, expr, when
from pyspark.sql.types import IntegerType
from pyspark.sql import Window

In [0]:
%run "./utils/create_schemas"

In [0]:
file_type = "json"

def read_s3(topic, file_type, schema):
    """Given a topic in a s3 bucket, returns a dataframe with the contents"""
    file_location = f"mnt/0e36c8cd403d_s3_bucket/topics/{topic}/partition=0/*.json"

    df = (spark.read.format(file_type)
          .schema(schema)
          .load(file_location)
    )

    return df

In [0]:
df_geo_pre_cleaning = read_s3('0e36c8cd403d.geo', file_type, geo_schema)
df_pin_pre_cleaning = read_s3('0e36c8cd403d.pin', file_type, pin_schema)
df_user_pre_cleaning = read_s3('0e36c8cd403d.user', file_type, user_schema)

In [0]:
%run "./utils/clean"

In [0]:
w = Window.partitionBy('country')

(
    df_pin.join(df_geo,
                on="ind",
                how="inner"
                )
    # Group by and add column category_count
    .groupBy("country", "category")
    .agg(F.count("category").alias("category_count"))

    # Get max category per country
    .withColumn('max_count', F.max('category_count').over(w))
    .where(F.col('category_count') == F.col('max_count'))
    .drop('max_count')
    .show(truncate=100)
)



+--------------------------------------------+--------------+--------------+
 country| category|category_count|
+--------------------------------------------+--------------+--------------+
 Afghanistan| education| 17|
 Albania| art| 30|
 Algeria| quotes| 40|
 American Samoa| tattoos| 12|
 Andorra| tattoos| 11|
 Angola|diy-and-crafts| 4|
 Angola| education| 4|
 Anguilla|diy-and-crafts| 6|
Antarctica (the territory South of 60 deg S)| christmas| 7|
 Antigua and Barbuda| art| 4|
 Antigua and Barbuda| travel| 4|
 Argentina| tattoos| 13|
 Armenia| beauty| 5|
 Armenia| art| 5|
 Aruba| christmas| 11|
 Australia| christmas| 3|
 Australia| mens-fashion| 3|
 Australia| travel| 3|
 Austria| travel| 6|
 Azerbaijan| finance| 9|
+--------------------------------------------+--------------+--------------+
only showing top 20 rows

In [0]:
w = Window.partitionBy("post_year")

(
    df_pin.join(df_geo,
                on="ind",
                how="inner"
    )
    .withColumn("post_year", F.year("timestamp"))
    .where(F.col("post_year").between("2018", "2022"))

    # Group by and add column category_count
    .groupBy("post_year", "category")
    .agg(F.count("category").alias("category_count"))

    # Get max category per year
    .withColumn("max_count", F.max("category_count").over(w))
    .where(F.col("category_count") == F.col("max_count"))
    .drop("max_count")
    .show()
)

+---------+---------+--------------+
post_year| category|category_count|
+---------+---------+--------------+
 2018|christmas| 44|
 2019| art| 34|
 2020|christmas| 34|
 2021| quotes| 36|
 2022|christmas| 40|
+---------+---------+--------------+

In [0]:
w = Window.partitionBy("country")

df_user_with_most_followers_by_country = (
    df_pin.join(df_geo,
                on="ind",
                how="inner"
                )
            # For each country return the user with the most followers
            .drop_duplicates(["country", "poster_name", "follower_count"])
            .select("country", "poster_name", "follower_count")
            .withColumn("max_follower_count", F.max("follower_count").over(w))
            .where(F.col("follower_count") == F.col("max_follower_count"))
            .drop("max_follower_count")
)

df_user_with_most_followers_by_country.show(truncate=100)

+--------------------------------------------+-----------------------------------------------------------------+--------------+
 country| poster_name|follower_count|
+--------------------------------------------+-----------------------------------------------------------------+--------------+
 Afghanistan| 9GAG| 3000000|
 Albania| The Minds Journal| 5000000|
 Algeria| Apartment Therapy| 5000000|
 American Samoa| Mamas Uncut| 8000000|
 Andorra| Teachers Pay Teachers| 1000000|
 Angola| Tastemade| 8000000|
 Anguilla| Hometalk.com| 5000000|
Antarctica (the territory South of 60 deg S)| Refinery29| 1000000|
 Antigua and Barbuda| Country Living Magazine| 1000000|
 Argentina| Cheezburger| 2000000|
 Armenia| Michelle {CraftyMorning.com}| 892000|
 Aruba| GQ Magazine| 874000|
 Australia| Mujer de 10 | 2000000|
 Austria| Allwomenstalk - girl rules | 813000|
 Azerbaijan| Style Me Pretty| 6000000|
 Bahamas| Handmade Charlotte| 948000|
 Bahrain| Rainy Day Mum| 337000|
 Bangladesh| Better Homes and Gardens| 4000000|
 Barbados| The Creativity Exchange| 410000|
 Belarus|Prudent Penny Pincher - Home Decor, Organization, Crafts, Recipes| 648000|
+--------------------------------------------+-----------------------------------------------------------------+--------------+
only showing top 20 rows

In [0]:

(
    df_user_with_most_followers_by_country
    .select("country", "follower_count")
    .sort("follower_count", "country", ascending=[False, True])
    .limit(1)
    .show()
)



+--------------+--------------+
 country|follower_count|
+--------------+--------------+
American Samoa| 8000000|
+--------------+--------------+

In [0]:
def add_age_group_column(df):
    """Add age_group column"""
    df = df.withColumn('age_group', 
                       when( (F.col("age") >= 18) & (F.col("age") < 25), "18-24")
                       .when( (F.col("age") >= 25) & (F.col("age") < 36), "25-35")
                       .when( (F.col("age") >= 36) & (F.col("age") < 51), "36-50")
                       .otherwise("+50")
               )

    return df

In [0]:

w = Window.partitionBy("age_group")

(
    df_pin.join(df_user,
                on="ind",
                how="inner")
    .transform(add_age_group_column)
    .groupby("age_group", "category")
    .agg(F.count("category").alias("category_count"))
    .withColumn("max_category_count", F.max("category_count").over(w))
    .where(F.col("category_count") == F.col("max_category_count"))
    .drop("max_category_count")
    .show()
)

+---------+---------+--------------+
age_group| category|category_count|
+---------+---------+--------------+
 +50| vehicles| 21|
 18-24| tattoos| 90|
 25-35|christmas| 59|
 36-50| finance| 41|
+---------+---------+--------------+

In [0]:
magic_percentile = F.expr('percentile_approx(follower_count, 0.5)')

(
    df_pin.join(df_user,
                on="ind",
                how="inner")
    .transform(add_age_group_column)
    .groupBy('age_group').agg(magic_percentile.alias('median_follower_count'))
    .show()
)

+---------+---------------------+
age_group|median_follower_count|
+---------+---------------------+
 36-50| 7000|
 +50| 1000|
 18-24| 130000|
 25-35| 22000|
+---------+---------------------+

In [0]:
(
    df_user.join(df_geo,
                 on="ind",
                 how="inner")
    .withColumn("joined_year", F.year("date_joined"))
    .where( ((F.col("joined_year") > 2014) & (F.col("joined_year") < 2021)) )
    .withColumn("post_year", F.year("timestamp"))
    .groupby("post_year").agg(F.count(F.col("post_year")).alias("number_users_joined"))
    .sort("post_year", ascending=True)
    .show()
)

+---------+-------------------+
post_year|number_users_joined|
+---------+-------------------+
 2017| 69|
 2018| 391|
 2019| 326|
 2020| 358|
 2021| 356|
 2022| 319|
+---------+-------------------+

In [0]:
magic_percentile = F.expr('percentile_approx(follower_count, 0.5)')

(
    df_user.join(df_pin,
                 on="ind",
                 how="inner")
    .join(df_geo,
          on="ind",
          how="inner")
    .withColumn("joined_year", F.year("date_joined"))
    .where( ((F.col("joined_year") > 2014) & (F.col("joined_year") < 2021)) )
    .withColumn("post_year", F.year("timestamp"))
    .groupBy('post_year').agg(magic_percentile.alias('median_follower_count'))
    .show()
)

+---------+---------------------+
post_year|median_follower_count|
+---------+---------------------+
 2018| 36000|
 2022| 34000|
 2019| 28000|
 2020| 30000|
 2017| 39000|
 2021| 20000|
+---------+---------------------+

In [0]:
magic_percentile = F.expr('percentile_approx(follower_count, 0.5)')

(
    df_user.join(df_pin,
                 on="ind",
                 how="inner")
    .join(df_geo,
          on="ind",
          how="inner")
    .withColumn("joined_year", F.year("date_joined"))
    .where( ((F.col("joined_year") > 2014) & (F.col("joined_year") < 2021)) )
    .withColumn("post_year", F.year("timestamp"))
    .transform(add_age_group_column)
    .groupBy('post_year', 'age_group').agg(magic_percentile.alias('median_follower_count'))
    .sort(["post_year", "age_group"], ascending=[True, True])
    .show()
)

java.lang.RuntimeException: abort: DriverClient destroyed
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$3(DriverClient.scala:577)
	at scala.concurrent.Future.$anonfun$flatMap$1(Future.scala:307)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:54)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:77)
	at com.databricks.threading.DatabricksExecutionContext$InstrumentedRunnable.run(DatabricksExecutionContext.scala:36)
	at com.databricks.threading.NamedExecutor$$anon$2.$anonfun$run$2(NamedExecutor.scala:378)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:420)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:418)
	at com.databricks.threading.NamedExecutor.withAttributionContext(NamedExecutor.scala:297)
	at com.databricks.threading.NamedExecutor$$anon$2.$anonfun$run$1(NamedExecutor.scala:376)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.context.integrity.IntegrityCheckContext$ThreadLocalStorage$.withValue(IntegrityCheckContext.scala:44)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:368)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)